# 🔍 Assignment: Build a Context-Aware LLM Agent Using LangChain

##  Objective
Design and implement your **own LLM-powered agent** using the LangChain ecosystem. This agent should:
- Use at least **one tool**
- Incorporate **function-calling**
- Integrate **LangSmith** for observability
- Implement a **fallback mechanism**
- Bonus: Use the **Agentic** library to define a clear multi-step workflow




```

```

##  Your Task
- Choose a use case (e.g., travel planner, medical assistant, customer support bot, product recommender, etc.)
- Build a custom tool (e.g., API wrapper, knowledge lookup, calculator)
- Use LangChain agents to manage interaction
- Integrate LangSmith to track runs
- Handle failure gracefully with a fallback approach
- Optionally add structured workflows with Agentic


In [ ]:
!pip install dotenv langchain pydantic langchain_openai
!pip install -U langchain-community
!pip install chromadb
!pip install google-search-results
!pip install serpapi
!pip install openweathermap
!pip install groq
!pip install langsmith
!pip install ChatGroq
!pip install -U langchain-groq
!pip install langchain-mistralai
!pip install langchain-core langsmith

In [17]:
import datetime
import io
import json
import os
import sys
import time
from google.colab import userdata
from contextlib import redirect_stdout
from enum import Enum
from typing import Annotated, Dict, List, Optional
import requests
from google.colab import userdata
from groq import Groq
from langchain.agents import (
    AgentExecutor,
    AgentType,
    create_tool_calling_agent,
    initialize_agent,
)
from langchain.callbacks import StdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI
from langsmith.run_helpers import traceable
from pydantic import BaseModel, Field
from serpapi.google_search import GoogleSearch

##  Step 2: Set Up Keys
 You’ll need:
- An OpenAI API key (for LLM calls)/ Groq API
- A LangSmith API key (for logging and observability)

In [3]:
groqKey = userdata.get('groq')
langsmithKey = userdata.get('Langsmith')
tavilyKey = userdata.get('tavily')
serpKey = userdata.get('serp')
openweatherKey = userdata.get('openweather')
mistralKey = userdata.get('mistral')

##  Step 3: Define Custom Tools
 Build at least one tool (e.g., API, calculator, mock database lookup). Use decorators or `Tool` wrappers.

In [4]:
# Enum for allowed event date ranges

class EventDate(str, Enum):
  """
  Represents predefined date ranges for event queries.
  The string values correspond to how they might be interpreted by an underlying API
  or internal logic.
  """
  TODAY = "today"
  TOMORROW = "tomorrow"
  THIS_WEEK = "week"
  THIS_WEEKEND = "weekend"
  NEXT_WEEK = "next_week"
  THIS_MONTH = "month"
  NEXT_MONTH = "next_month"

# --- Define the Events Query Pydantic Model ---

class EventsQuery(BaseModel):
  """
  A Pydantic model for querying events with structured parameters.
  This model defines the expected input for a function that lists events.
  """

  event_type: str = Field(...,description="What type of event are you looking for? Example: Party,food,tourism, etc.")
  event_date: EventDate = Field(
      ...,
      description=(
          "The specific date range for the events. "
          "Must be one of the predefined values: "
          f"{', '.join([f'`{e.value}`' for e in EventDate])}."
      )
  )
  location: str = Field(...,description="The location of the event. This field is required.")


@tool("event_getter_tool", args_schema=EventsQuery, description="Fetches specific events at a given time period from now in a given location")
def get_events_from_serpapi(event_type: str, event_date: EventDate, location: str) -> List[Dict]:
  """
  Fetches events using SerpAPI based on the provided parameters.
  """

  #print(f"\n[TOOL CALL: get_events] Calling SerpAPI for: '{event_type} in {location}' on date: {event_date.value}...")

  htichips_list = [f"date:{event_date.value}"]
  htichips_param = ",".join(htichips_list)

  main_query = f"{event_type} in {location}"

  #Parameters for the google search
  params = {
      "api_key": serpKey,
      "engine": "google_events",
      "q": main_query,
      "hl": "en",
      "gl": "us",
      "htichips": htichips_param,
  }

  try:
    search = GoogleSearch(params)
    results = search.get_dict()
    events_results = results.get("events_results", [])
    events_results = events_results[:1] # Limit results for now to minimize tokens

    if events_results:
        print(f"[TOOL SUCCESS] Fetched {len(events_results)} events for '{event_type}' in {location}.")
    else:
        print(f"[TOOL INFO] No events found for '{event_type}' in {location} from SerpAPI.")
    return events_results

  except Exception as e:
    print(f"[TOOL ERROR] An error occurred during SerpAPI call: {e}")
    return []

In [5]:
class AirPollutionQuery(BaseModel):
  """
  A Pydantic model for querying current air pollution data for a city.
  """
  city: str = Field(..., description="The single city name for which to get air pollution data.")

@tool("get_current_air_pollution", args_schema=AirPollutionQuery,
  description="Fetches the most recent air pollution data (AQI, CO, NO2, O3, PM2.5, etc.) for a specified city.")
def get_current_air_pollution_data_by_city(city: str) -> Dict:
  """
  Fetches the most recent air pollution data for a given city name.
  """

  #print(f"\n[TOOL CALL: get_current_air_pollution] Getting air pollution for city: {city}...")

  # --- Step 1: Get Latitude and Longitude using Geocoding API ---
  geocode_url = f'http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={openweatherKey}'
  lat = None
  lon = None

  try:
    geocode_response = requests.get(geocode_url)
    geocode_response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    geocode_data = geocode_response.json()

    if geocode_data and len(geocode_data) > 0:
      lat = geocode_data[0].get('lat')
      lon = geocode_data[0].get('lon')
      print(f"Found coordinates for {city}: Lat={lat}, Lon={lon}")
    else:
      print(f"[TOOL ERROR] Could not find coordinates for city: {city}.")
      return {"error": f"Could not find coordinates for city: {city}. Please check the city name."}

  except requests.exceptions.RequestException as req_err:
      print(f'[TOOL ERROR] Geocoding API request error occurred: {req_err}')
      return {"error": f"Geocoding API error: {req_err}"}
  except json.JSONDecodeError:
      print('[TOOL ERROR] Error: Could not decode JSON from Geocoding API response.')
      return {"error": "Invalid JSON response from Geocoding API."}
  except Exception as e:
      print(f'[TOOL ERROR] An unexpected error occurred during geocoding: {e}')
      return {"error": f"Unexpected geocoding error: {e}"}

  # --- Step 2: Get Current Air Pollution Data using obtained coordinates ---

  if lat is None or lon is None:
      print("[TOOL ERROR] Latitude or Longitude not available after geocoding, cannot fetch air pollution data.")
      return {"error": "Latitude or Longitude not available to fetch air pollution data."}

  pollution_url = (
      f"http://api.openweathermap.org/data/2.5/air_pollution?"
      f"lat={lat}&lon={lon}&appid={openweatherKey}"
  )

  try:
    pollution_response = requests.get(pollution_url)
    pollution_response.raise_for_status()
    pollution_data = pollution_response.json()

    if pollution_data and 'list' in pollution_data and len(pollution_data['list']) > 0:

      most_recent_data = pollution_data['list'][0]
      timestamp = most_recent_data.get('dt')

      if timestamp:
        dt_object = datetime.datetime.fromtimestamp(timestamp, datetime.timezone.utc)
        most_recent_data['datetime_utc'] = dt_object.isoformat()

      print(f"[TOOL SUCCESS] Fetched air pollution for {city}.")

      return most_recent_data

    else:
      print("[TOOL INFO] No current air pollution data found for this location or unexpected response format.")
      return {"error": "No air pollution data found or unexpected response."}

  except requests.exceptions.RequestException as req_err:
    print(f'[TOOL ERROR] Air Pollution API request error occurred: {req_err}')
    return {"error": f"Air Pollution API error: {req_err}"}
  except json.JSONDecodeError:
    print('[TOOL ERROR] Error: Could not decode JSON from Air Pollution API response.')
    return {"error": "Invalid JSON response from Air Pollution API."}
  except Exception as e:
    print(f'[TOOL ERROR] An unexpected error occurred during air pollution data fetch: {e}')
    return {"error": f"Unexpected air pollution fetch error: {e}"}

In [ ]:
agent_tools = [
    get_events_from_serpapi,
    get_current_air_pollution_data_by_city
    ]

##  Step 4: Enable Function-Calling
 Define structured functions for your agent to call. Use `openai.FunctionsAgent` or LangChain's `OPENAI_FUNCTIONS` agent type.

In [ ]:
@traceable(name="VacationPlanningRun")
def run(input_text: str):
  f = io.StringIO()
  with redirect_stdout(f):  # Suppress stdout temporarily
      result = agent_executor.invoke({"input": input_text})
  return result["output"]

In [6]:
llm = ChatGroq(
        model="llama3-70b-8192", # Recommended Groq model for tool use
        temperature=0,
        max_retries=3,
        api_key=groqKey
    )

llm_with_tools = llm.bind_tools(agent_tools)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that helps people find events in a city and check for air pollution."),
        MessagesPlaceholder(variable_name="chat_history", optional=True), # For conversational memory
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),            # Agent's internal thoughts and tool outputs
    ]
)

##  Step 5: Integrate LangSmith
 Wrap your LLM or agent with LangSmith to observe performance and execution flows.

In [7]:
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langsmithKey                         # Update with your API key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
project_name = "Assignment_2"                                          # Update with your project name
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["GROQ_API_KEY"] = groqKey

##  Step 6: Fallback Strategy
 Define a fallback agent or tool to be used when the main tool or agent fails.

In [8]:
# Define a fallback LLM using ChatMistralAI
fallback_llm = ChatMistralAI(
    model="mistral-small-2501",
    temperature=0,
    api_key=mistralKey
)

llm_with_fallbacks = llm_with_tools.with_fallbacks([fallback_llm])

In [20]:
# 7. Create the tool-calling agent
agent = create_tool_calling_agent(llm_with_fallbacks, agent_tools, prompt)

memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="output",
        return_messages=True       # Return history as a list of message objects
    )

# 8. Create an AgentExecutor to run the agent
agent_executor = AgentExecutor(
    agent=agent,
    tools=agent_tools,            # Provide the list of tools to the executor
    verbose=False,                # See the agent's thinking process
    handle_parsing_errors=True,
    memory=memory,
    return_intermediate_steps=True
)

In [21]:
print(run("Find me events in Tokyo for next month"))

Here are the events in Tokyo for next month:

* Baki Exhibition: June 13, 10:30 AM – July 1, 9:00 PM GMT+9 at SHIBUYA TSUTAYA, Q Front, B2F-8F 21-6 Udagawacho, Shibuya, Tokyo, Japan.

Let me know if you need more information or if there's anything else I can help you with!


##  (Bonus) Step 7: Define a Workflow using Agentic
 Use the `agentic` library to define a step-by-step flow your agent follows.

 Submission Checklist
- [ ] Agent runs end-to-end on your chosen use case
- [ ] At least one custom tool is integrated
- [ ] Function-calling works with tools
- [ ] LangSmith integration logs all calls
- [ ] Fallback logic is demonstrated
- [ ] Bonus: Agentic workflow is defined

##  Grading (100 pts )


##  Tip
You are free to choose **any use case** — be creative but keep your design modular and well-commented!